In [ ]:
import pandas as pd
import numpy as nu
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

In [ ]:
Tourism_df = pd.read_csv('Dataset_Tourism_final.csv')

In [ ]:
Tourism_df.head()

In [ ]:
Tourism_df.isnull().sum()

In [ ]:
Tourism_df.dropna(inplace=True)

In [ ]:
Tourism_df.isnull().sum()

In [ ]:
Tourism_df.shape

In [ ]:
Tourism_df.nunique()

In [ ]:
Tourism_df.info()

In [ ]:
# IMPORTING SMOTE FOR HANDLING IMBALANCED DATA AND ENCODERS FOR FEATURE TRANSFORMATION  
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,StandardScaler
from category_encoders import TargetEncoder
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
Tourism_df.to_csv('Tourism_visitmode.csv', index=False)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE
from category_encoders import TargetEncoder

# Feature selection
selected_features = ["VisitYear", "VisitMonth",
                     "ContinentId", "CountryId", "Country", "Attraction", "AttractionType"]

X = Tourism_df[selected_features].copy()
y = Tourism_df["VisitModeName"]


In [ ]:
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import joblib

# Label encode target
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(Tourism_df["VisitModeName"])

# Target encode Attraction and Country separately
enc_attraction = TargetEncoder()
X["Attraction"] = enc_attraction.fit_transform(X["Attraction"], y)

enc_country = TargetEncoder()
X["Country"] = enc_country.fit_transform(X["Country"], y)

# One-hot encode AttractionType
categorical_features = ["AttractionType"]
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_features = ohe.fit_transform(X[categorical_features])
encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(categorical_features))
encoded_df.index = X.index

# Drop original and concat encoded
X = X.drop(columns=categorical_features)
X = pd.concat([X, encoded_df], axis=1)

# # SMOTE
# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(X, y)

# # Combine to final dataset
# y_smote = pd.Series(y_smote, name='VisitModeName')
# balanced_tourism_data = pd.concat([X_smote, y_smote], axis=1)

# # Check distribution
# print("Before SMOTE:")
# print(pd.Series(y).value_counts())

# print("\nAfter SMOTE:")
# print(y_smote.value_counts())


In [ ]:
final_features = X.columns.tolist()
import joblib
joblib.dump(final_features, "feature_columns.pkl")

In [ ]:
# SMOTE
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

# Combine to final dataset
y_smote = pd.Series(y_smote, name='VisitModeName')
balanced_tourism_data = pd.concat([X_smote, y_smote], axis=1)

# Check distribution
print("Before SMOTE:")
print(pd.Series(y).value_counts())

print("\nAfter SMOTE:")
print(y_smote.value_counts())


In [ ]:
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# from imblearn.over_sampling import SMOTE
# from category_encoders import TargetEncoder

# # Feature selection
# selected_features = ["UserId", "VisitYear", "VisitMonth", "AttractionId", 
#                      "ContinentId", "RegionId", "CountryId", "Attraction", "AttractionType"]

# X = Tourism_df[selected_features].copy()
# y = Tourism_df["VisitModeName"]

# # One-Hot Encode only AttractionType
# categorical_features = ["AttractionType"]

# ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
# encoded_features = ohe.fit_transform(X[categorical_features])
# encoded_df = pd.DataFrame(encoded_features, columns=ohe.get_feature_names_out(categorical_features))
# encoded_df.index = X.index

# # Label Encoding for target variable (VisitModeName)
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(Tourism_df["VisitModeName"])  # Already numeric

# # Target Encode Attraction
# target_enc = TargetEncoder()
# X["Attraction"] = target_enc.fit_transform(X["Attraction"], y)

# # Drop original categorical and add encoded
# X = X.drop(columns=categorical_features)
# X = pd.concat([X, encoded_df], axis=1)

# # SMOTE
# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(X, y)

# # Combine to final dataset
# y_smote = pd.Series(y_smote, name='VisitModeName')
# balanced_tourism_data = pd.concat([X_smote, y_smote], axis=1)

# # Check distribution
# print("Before SMOTE:")
# print(pd.Series(y).value_counts())

# print("\nAfter SMOTE:")
# print(y_smote.value_counts())



In [ ]:
# import pandas as pd
# import joblib
# from category_encoders import TargetEncoder
# from sklearn.preprocessing import LabelEncoder

# # Load your dataset
# Tourism_df = pd.read_csv("Tourism_visitmode.csv")

# # Convert target (VisitModeName) to numeric using LabelEncoder
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(Tourism_df["VisitModeName"])

# # Save label encoder for prediction decoding later
# joblib.dump(label_encoder, "label_encoder.pkl")

# # ----- Target Encoder for Attraction -----
# enc_attraction = TargetEncoder()
# enc_attraction.fit(Tourism_df[["Attraction"]], y)
# joblib.dump(enc_attraction, "target_enc_attraction.pkl")

# # ----- Target Encoder for Country -----
# enc_country = TargetEncoder()
# enc_country.fit(Tourism_df[["Country"]], y)
# joblib.dump(enc_country, "target_enc_country.pkl")


In [ ]:
#  Step 4: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

In [ ]:
# Feature Scaling (Standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Train Decision Tree Classifier with max depth of 5
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

# Predict using the trained Decision Tree model
dt_pred = dt_model.predict(X_test)
print(dt_pred)

In [ ]:
#  Step 8: Model Evaluation
accuracy = accuracy_score(y_test, dt_pred)
precision = precision_score(y_test, dt_pred, average='weighted')
recall = recall_score(y_test, dt_pred, average='weighted')
f1 = f1_score(y_test, dt_pred, average='weighted')

print(f" Accuracy: {accuracy:.2f}")
print(f" Precision: {precision:.2f}")
print(f" Recall: {recall:.2f}")
print(f" F1 Score: {f1:.2f}")

In [ ]:
# Creates and trains a Random Forest classifier with 100 trees, max depth of 5, and fixed random state.  
rf_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rf_model.fit(X_train, y_train)

# Makes predictions on the test data (X_test) using the trained Random Forest model.  
rf_pred = rf_model.predict(X_test)
rf_pred

# Computes evaluation metrics for the model: accuracy, precision, and recall using the test labels (y_test) and predictions (rf_pred).
# Prints the results formatted to two decimal places. 
accuracy = accuracy_score(y_test, rf_pred)
precision = precision_score(y_test, rf_pred, average='weighted')
recall = recall_score(y_test, rf_pred, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, rf_pred, target_names=label_encoder.classes_))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, rf_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

In [ ]:
# Creates an XGBoost classifier with 50 trees, max depth of 3, and a learning rate of 0.2 for optimized training.  
# Trains the model on the training data (X_train, y_train) and makes predictions on the test data (X_test).  

xgb_model = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.5,subsample=0.8, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# Computes accuracy, precision, and recall for the XGBoost model using test labels (y_test) and predictions (xgb_pred).  
# Prints the results formatted to two decimal places.  
accuracy = accuracy_score(y_test, xgb_pred)
precision = precision_score(y_test, xgb_pred, average='weighted')
recall = recall_score(y_test, xgb_pred, average='weighted')
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
# Save all encoders and model
joblib.dump(enc_attraction, "target_enc_attraction(class).pkl")
joblib.dump(enc_country, "target_enc_country(class).pkl")
joblib.dump(ohe, "onehot_encoder(class).pkl")
joblib.dump(label_encoder, "label_encoder(class).pkl")
joblib.dump(xgb_model, "xgboost_model(class).pkl")

In [ ]:
# import joblib

# # Saving model
# joblib.dump(target_enc,'Target Encoder Model(class).pkl')
# print("Target encoded model saved")

# joblib.dump(ohe,'One-Hot Endcoder Model(class).pkl')
# print("One-Hot Encoded model saved")

# joblib.dump(label_encoder,'label_encoder(class).pkl')
# print("Lable encoded has been saved")

# joblib.dump(xgb_model,'XGBoost model(class).pkl')
# print("Best XGBoost Model has been saved")